In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
#import seaborn as sns
import os
import random
import cv2
import json
from sklearn.model_selection import train_test_split
from torch import nn
import constants as c
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




In [5]:
# Load data into dataframe

train_dir = r'C:\Users\User\Gender_classification_from_voice\data\json_french_data'

map_characters = {0: 'M', 1: 'W'}

labels_dict = {'M': 0, 'W': 1}

order_list = ('M', 'W')

def load_data():
    """
    Loads data and preprocess. Returns train and test data along with labels.
    """
    images = []
    labels = []
    size = 29,449
    num=0
    count=0

    print("LOADING DATA FROM : ",end = "")


    # this code for singal language load
    for folder in os.listdir(train_dir): ## to check 
        if folder[0] == '.':
            continue
        print(folder, end = ' | ')
        for json_file in os.listdir(train_dir + "/" + folder):
            temp_json_file = json.load(open(train_dir + '/' + folder + '/' + json_file))
            temp_np_array = np.array(temp_json_file)
            temp = [cv2.resize(temp_np_array[0], size)]
            temp_img = np.array(temp)
            # plt.imshow(cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB))
            # plt.show()
            images.append(temp_img)
            labels.append(num)
            count+=1 #count the number of photos
            if count% 5000==0:
                print("data number: ",count ,"was load")
        num+=1
    
    
    images = np.array(images)
    # images = images.astype('float32')/255
    
    
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.3, random_state=42)
    X_test, X_validation, Y_test, Y_validation = train_test_split(X_test, Y_test, test_size = 0.8,random_state=42)
    
    
    print()
    print('Loaded', len(X_train),'images for training,','Train data shape =',X_train.shape)
    print('Loaded', len(X_validation),'images for validation','validation data shape =',X_validation.shape)
    print('Loaded', len(X_test),'images for testing','Test data shape =',X_test.shape)
    print('\n')

    
    return X_train, X_test, Y_train, Y_test, X_validation, Y_validation

X_train, X_test, Y_train, Y_test, X_validation, Y_validation= load_data()

LOADING DATA FROM : M | [[[10.05185223  0.96224552 -0.93897057 ... -5.73374462 -6.5121913
   -7.42086411]
  [10.07750511  0.93062168 -0.93264818 ... -5.71655512 -6.50114918
   -7.42629004]
  [10.08192635  0.88658017 -0.8849135  ... -5.68826103 -6.50071621
   -7.41243839]
  ...
  [ 9.90593433  3.93466592 -0.14956239 ... -7.40651941 -8.24702549
   -6.95793247]
  [10.26316357  4.5300312  -0.48362666 ... -7.33768845 -8.08211231
   -6.46876574]
  [10.78181553  3.50254369 -0.78426397 ... -6.65375662 -7.66215944
   -7.15288734]]]
[array([[10.05185223,  0.96224552, -0.93897057, ..., -5.73374462,
        -6.5121913 , -7.42086411],
       [10.07750511,  0.93062168, -0.93264818, ..., -5.71655512,
        -6.50114918, -7.42629004],
       [10.08192635,  0.88658017, -0.8849135 , ..., -5.68826103,
        -6.50071621, -7.41243839],
       ...,
       [ 9.90593433,  3.93466592, -0.14956239, ..., -7.40651941,
        -8.24702549, -6.95793247],
       [10.26316357,  4.5300312 , -0.48362666, ..., -7.337

KeyboardInterrupt: 

In [ ]:
# The model

NUM_OF_SPEAKERS = 2
DROP_OUT = 0.5

class Convolutional_Speaker_Identification(nn.Module):

    def cal_paddind_shape(self, new_shape, old_shape, kernel_size, stride_size):
        return (stride_size * (new_shape - 1) + kernel_size - old_shape) / 2

    def __init__(self):
        super().__init__()
        self.conv_2d_1 = nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=2)
        self.bn_1 = nn.BatchNorm2d(96)
        self.max_pool_2d_1 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))

        self.conv_2d_2 = nn.Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=2)
        self.bn_2 = nn.BatchNorm2d(256)
        self.max_pool_2d_2 = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))

        self.conv_2d_3 = nn.Conv2d(256, 384, kernel_size=(3, 3), padding=2)
        self.bn_3 = nn.BatchNorm2d(384)

        self.conv_2d_4 = nn.Conv2d(384, 256, kernel_size=(3, 3), padding=2)
        self.bn_4 = nn.BatchNorm2d(256)

        self.conv_2d_5 = nn.Conv2d(256, 256, kernel_size=(3, 3), padding=2)
        self.bn_5 = nn.BatchNorm2d(256)
        self.max_pool_2d_3 = nn.MaxPool2d(kernel_size=(5, 3), stride=(3, 2))

        self.conv_2d_6 = nn.Conv2d(256, 4096, kernel_size=(9, 1), padding=0)
        self.drop_1 = nn.Dropout(p=DROP_OUT)

        self.global_avg_pooling_2d = nn.AdaptiveAvgPool2d((1, 1))
        self.dense_1 = nn.Linear(4096, 1024)
        self.drop_2 = nn.Dropout(p=DROP_OUT)

        self.dense_2 = nn.Linear(1024, NUM_OF_SPEAKERS)

    def forward(self, X):
        x = nn.ReLU()(self.conv_2d_1(X))
        x = self.bn_1(x)
        x = self.max_pool_2d_1(x)

        x = nn.ReLU()(self.conv_2d_2(x))
        x = self.bn_2(x)
        x = self.max_pool_2d_2(x)

        x = nn.ReLU()(self.conv_2d_3(x))
        x = self.bn_3(x)

        x = nn.ReLU()(self.conv_2d_4(x))
        x = self.bn_4(x)

        x = nn.ReLU()(self.conv_2d_5(x))
        x = self.bn_5(x)
        x = self.max_pool_2d_3(x)

        x = nn.ReLU()(self.conv_2d_6(x))
        x = self.drop_1(x)
        x = self.global_avg_pooling_2d(x)

        x = x.view(-1, x.shape[1])  # output channel for flatten before entering the dense layer
        x = nn.ReLU()(self.dense_1(x))
        x = self.drop_2(x)

        x = self.dense_2(x)
        y = nn.LogSoftmax(dim=1)(x)   # consider using Log-Softmax

        return y

    # def get_epochs(self):
    #     return epochs
    # def get_learning_rate(self):
    #     return lr
    # def get_batch_size(self):
    #     return batch_size
    # def to_string(self):
    #     return "Convolutional_Speaker_Identification_Log_Softmax_Model-epoch_"
    
    

In [ ]:
# prepare the data,model

import torch.nn as nn
import torch
import torch.optim as optim

# Define loss, optimizer and metrics
model = Convolutional_Speaker_Identification()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device( "cpu")
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 32
epochs = 30

model_save_name='model_french.pt'

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("using cuda")
else:
    print("not using cuda")
    device = torch.device("cpu")

data_x = torch.tensor(X_train).to(device)
data_y = torch.tensor(Y_train).to(device)

val_x = torch.tensor(X_validation).to(device)
val_y = torch.tensor(Y_validation).to(device)


# Define your training data loader
dataset = torch.utils.data.TensorDataset(data_x, data_y)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define your validation data loader
validation_dataset = torch.utils.data.TensorDataset(val_x, val_y)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

model = model.to(torch.float)

In [ ]:
# train and validation of the model

best_validation_accuracy = -1.0

# Training loop
for epoch in range(epochs):
    # Training phase
    model.train()  # Set the model to training mode
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        # Forward pass
        images = images.to(torch.float)
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = correct / total
    train_loss /= len(train_loader)

    print(f"Epoch {epoch+1}/{epochs}:")
    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    validation_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.to(torch.float)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    validation_accuracy = correct / total
    validation_loss /= len(validation_loader)
    print(f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}")
    print()
    
    # save the best model and test in th end
    if best_validation_accuracy < validation_accuracy :
        best_validation_accuracy = validation_accuracy
        print(f"best model now with val_acc: {best_validation_accuracy:.4f}")
        torch.save(model.state_dict(), model_save_name)
    print()


    
    

In [ ]:
# keep Training,validation with smaller lr

optimizer = optim.Adam(model.parameters(), lr=0.0001)
extra_epochs=15

for epoch in range(extra_epochs):
    # Training phase
    model.train()  # Set the model to training mode
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        # Forward pass
        images = images.to(torch.float)
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = correct / total
    train_loss /= len(train_loader)

    print(f"Epoch {epoch+1}/{extra_epochs}:")
    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    validation_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images = images.to(torch.float)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    validation_accuracy = correct / total
    validation_loss /= len(validation_loader)
    print(f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}")
    print()
    
    # save the best model and test in th end
    if best_validation_accuracy < validation_accuracy :
        best_validation_accuracy = validation_accuracy
        print(f"best model now with val_acc: {best_validation_accuracy:.4f}")
        torch.save(model.state_dict(), model_save_name)
    print()

In [ ]:
#Test our best model

# Load the model
model = Convolutional_Speaker_Identification()
model_path = model_save_name  # Replace with the actual path to the saved model
model.load_state_dict(torch.load(model_path))
model = model.to(device)

test_x = torch.tensor(X_test).to(device)
test_y = torch.tensor(Y_test).to(device)

# Define your test data loader
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

#start test
model.eval()
with torch.no_grad():
    true_labels = []
    predicted_labels = []

    for images, labels in test_loader:
        images = images.to(torch.float)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        true_labels.extend(labels.cpu().tolist())
        predicted_labels.extend(predicted.cpu().tolist())

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

# Create a dictionary to store the results
results = {
    "model name": model_save_name,
    # 'best_val':f"{best_validation_accuracy:.4f}",
    "accuracy": f"{accuracy:.4f}",
    "precision": f"{precision:.4f}",
    "recall": f"{recall:.4f}",
    "f1": f"{f1:.4f}"
}

# Save the dictionary to a text file
with open("save.txt", "a") as file:
    file.write(str(results) + "\n")
